In [22]:
import pandas as pd
import numpy as np
import re
final_data = pd.read_csv('final_data.csv')
battle_result = pd.read_csv('battle_result.csv')


In [23]:
final_data

,event_map,battle_type,mode,team 1 brawler 1,team 1 brawler 1 power,team 1 brawler 1 trophies,team 1 brawler 2,team 1 brawler 2 power,team 1 brawler 2 trophies,team 1 brawler 3,...,team 1 brawler 3 trophies,team 2 brawler 1,team 2 brawler 1 power,team 2 brawler 1 trophies,team 2 brawler 2,team 2 brawler 2 power,team 2 brawler 2 trophies,team 2 brawler 3,team 2 brawler 3 power,team 2 brawler 3 trophies
0,Deep Diner,ranked,gemGrab,GUS,11.0,717.0,CROW,11.0,791.0,DARRYL,...,701.0,MORTIS,11.0,791.0,FANG,11.0,740.0,JESSIE,10.0,682.0
1,Sneaky Fields,ranked,brawlBall,RICO,11.0,782.0,NITA,11.0,837.0,EL PRIMO,...,890.0,MORTIS,11.0,799.0,EMZ,10.0,784.0,FRANK,11.0,905.0
2,Sneaky Fields,ranked,brawlBall,POCO,11.0,673.0,JACKY,10.0,681.0,SPIKE,...,704.0,GALE,11.0,651.0,SURGE,11.0,649.0,DARRYL,11.0,752.0
3,Sneaky Fields,ranked,brawlBall,LEON,10.0,641.0,SANDY,10.0,604.0,BIBI,...,663.0,BIBI,11.0,654.0,JACKY,10.0,625.0,CROW,11.0,674.0
4,Sneaky Fields,ranked,brawlBall,COLETTE,7.0,403.0,BUSTER,11.0,419.0,GENE,...,411.0,MAX,8.0,395.0,COLETTE,7.0,390.0,FANG,7.0,446.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256420,Goldarm Gulch,ranked,knockout,JACKY,10.0,829.0,SURGE,10.0,756.0,DARRYL,...,861.0,MORTIS,11.0,791.0,SHELLY,10.0,799.0,RUFFS,10.0,765.0
256421,Goldarm Gulch,ranked,knockout,RICO,11.0,792.0,MORTIS,11.0,800.0,TICK,...,795.0,MAX,11.0,826.0,TICK,10.0,829.0,RICO,11.0,812.0
256422,Goldarm Gulch,ranked,knockout,CROW,11.0,872.0,PIPER,11.0,910.0,STU,...,888.0,BELLE,11.0,899.0,PIPER,10.0,908.0,MAX,11.0,805.0
256423,Goldarm Gulch,ranked,knockout,BEA,11.0,14.0,PENNY,11.0,16.0,FANG,...,16.0,PIPER,11.0,12.0,CARL,11.0,15.0,GRAY,11.0,15.0


In [24]:
# Columns to be linked
linked_columns = [
    'team 1 brawler 1', 'team 1 brawler 2', 'team 1 brawler 3',
    'team 2 brawler 1', 'team 2 brawler 2', 'team 2 brawler 3'
]
# Dictionary to store the mappings
mappings = {}

# Concatenate the linked columns into a single series
concatenated = pd.concat([final_data[col] for col in linked_columns])

# Factorize the concatenated series
factorized_values, unique_values = pd.factorize(concatenated)

# Store the mappings for the linked columns
mappings['Brawlers'] = dict(enumerate(unique_values))

# Apply the factorized values back to the original linked columns
for i, col in enumerate(linked_columns):
    final_data[col] = factorized_values[i * len(final_data):(i + 1) * len(final_data)]

# Enumerate values in each remaining text column and store the mappings
for column in final_data.columns:
    if column not in linked_columns and final_data[column].dtype == 'object':
        final_data[column], unique_values = pd.factorize(final_data[column])
        mappings[column] = dict(enumerate(unique_values))

# Print the first value in the first column
print(final_data.iloc[0, 0])

# Print the mappings
print(mappings)

0
{'Brawlers': {0: 'GUS', 1: 'RICO', 2: 'POCO', 3: 'LEON', 4: 'COLETTE', 5: 'ROSA', 6: 'TICK', 7: 'JANET', 8: 'MR. P', 9: 'NITA', 10: 'EL PRIMO', 11: 'FANG', 12: 'BONNIE', 13: 'MANDY', 14: 'PIPER', 15: 'SPIKE', 16: '8-BIT', 17: 'JESSIE', 18: 'BEA', 19: 'DYNAMIKE', 20: 'BARLEY', 21: 'GRAY', 22: 'ASH', 23: 'STU', 24: 'COLT', 25: 'DARRYL', 26: 'MORTIS', 27: 'FRANK', 28: 'BIBI', 29: 'GRIFF', 30: 'SQUEAK', 31: 'CHESTER', 32: 'GENE', 33: 'MAX', 34: 'JACKY', 35: 'OTIS', 36: 'AMBER', 37: 'EMZ', 38: 'LOU', 39: 'SANDY', 40: 'BUSTER', 41: 'RUFFS', 42: 'PAM', 43: 'GROM', 44: 'BROCK', 45: 'BUZZ', 46: 'SPROUT', 47: 'BO', 48: 'CROW', 49: 'SURGE', 50: 'BULL', 51: 'NANI', 52: 'EVE', 53: 'BYRON', 54: 'SAM', 55: 'SHELLY', 56: 'GALE', 57: 'TARA', 58: 'CARL', 59: 'PENNY', 60: 'BELLE', 61: 'EDGAR', 62: 'LOLA', 63: 'MEG'}, 'event_map': {0: 'Deep Diner', 1: 'Sneaky Fields', 2: 'Flaring Phoenix', 3: 'Pinhole Punt', 4: 'Double Swoosh', 5: 'Open Zone', 6: 'Temple Ruins', 7: 'Super Beach', 8: 'Crystal Arcade', 9:

In [25]:
# List of columns to check for -1.0 values
columns_to_check = [
    'team 1 brawler 1 power', 'team 1 brawler 1 trophies',
    'team 1 brawler 2 power', 'team 1 brawler 2 trophies',
    'team 1 brawler 3 power', 'team 1 brawler 3 trophies',
    'team 2 brawler 1 power', 'team 2 brawler 1 trophies',
    'team 2 brawler 2 power', 'team 2 brawler 2 trophies',
    'team 2 brawler 3 power', 'team 2 brawler 3 trophies'
]


# Identify rows with -1.0 values in the specified columns
rows_to_remove = final_data[columns_to_check].isin([-1.0]).any(axis=1)

# Remove these rows from final_data
final_data= final_data[~rows_to_remove]

# Remove the same rows from battle_result
battle_result = battle_result[~rows_to_remove]


In [26]:
assert len(final_data) == len(battle_result), "DataFrames have different number of rows after cleaning."


In [27]:
# Factorize the values in the column
battle_result['battle_result'], unique_values = pd.factorize(battle_result['battle_result'])

# Print the enumerated battle_result DataFrame
print(battle_result)

# Check how many distinct values there are in the column
distinct_values_count = battle_result['battle_result'].nunique()

# Print the number of distinct values in the column
print(f"\nNumber of distinct values in 'result': {distinct_values_count}")

print("number of 1s in battle result",battle_result['battle_result'].value_counts()[1])
print("number of 0s in battle result",battle_result['battle_result'].value_counts()[0])

        battle_result
0                   0
1                   0
2                   0
3                   0
4                   1
...               ...
256420              1
256421              1
256422              1
256423              0
256424              0

[255545 rows x 1 columns]

Number of distinct values in 'result': 2
number of 1s in battle result 153995
number of 0s in battle result 101550


In [28]:
# Add a unique identifier to preserve alignment between battle_result and final_data
battle_result['original_index'] = battle_result.index
final_data['original_index'] = final_data.index

# Separate rows with 1 and 0 in battle_result
ones = battle_result[battle_result['battle_result'] == 1]
zeros = battle_result[battle_result['battle_result'] == 0]

# Get the count of rows with 0
num_zeros = len(zeros)

# Randomly sample rows with 1 to match the count of rows with 0
ones_sampled = ones.sample(n=num_zeros, random_state=42)

# Combine the sampled rows with 0 rows
battle_result = pd.concat([zeros, ones_sampled])

# Use the 'original_index' to filter final_data
final_data = final_data[final_data['original_index'].isin(battle_result['original_index'])]

# Drop the 'original_index' column from both DataFrames
battle_result = battle_result.drop(columns=['original_index']).reset_index(drop=True)
final_data = final_data.drop(columns=['original_index']).reset_index(drop=True)

# Print or save the balanced final_data
battle_result

C:\Users\ellio\AppData\Local\Temp\ipykernel_17288\3508247976.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['original_index'] = final_data.index


,battle_result
0,0
1,0
2,0
3,0
4,0
...,...
203095,1
203096,1
203097,1
203098,1


In [29]:
assert len(final_data) == len(battle_result), "DataFrames have different number of rows after cleaning."


In [30]:
print("number of 1s in battle result",battle_result['battle_result'].value_counts()[1])
print("number of 0s in battle result",battle_result['battle_result'].value_counts()[0])

number of 1s in battle result 101550
number of 0s in battle result 101550


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_data, battle_result, test_size=0.2, random_state=42)
X_test, X_eval, y_test, y_eval = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

assert len(X_test) == len(y_test), "DataFrames have different number of rows after cleaning."

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()
y_eval = y_eval.values.ravel()

In [32]:
y_train

array([0, 1, 0, ..., 1, 1, 1])

In [33]:
# Naive bayes model 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)
accuracy_eval = accuracy_score(y_test, y_pred_test)
print(f"Accuracy: {accuracy_eval}")

Accuracy: 0.5147218119153126


In [34]:
# logistic regression model
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# fit the model with data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

accuracy_eval = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_eval}")

Accuracy: 0.5192516001969473


c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
# Random forest model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=16)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_eval = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_eval}")

Accuracy: 0.7534711964549483


In [36]:
# neural network model
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(random_state=16)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_eval = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_eval}")

Accuracy: 0.5499753815854259


In [16]:
num_unique_values = final_data.iloc[:, 3].nunique()
print(f"Number of different values in the fourth column: {num_unique_values}")

Number of different values in the fourth column: 64


In [3]:
num_nan_event_map = final_data['mode'].isna().sum()
print(f"\nNumber of NaN values in 'event_map': {num_nan_event_map}")


Number of NaN values in 'event_map': 0


In [3]:
import re

# The input string
input_string = "[[{'tag': '#LGC0PQ98', 'name': '🍁PassØut™', 'brawler': {'id': 16000029, 'name': 'BEA', 'power': 11, 'trophies': 14}}, {'tag': '#VULUQUY8', 'name': '金|𝔻ανσχ💔', 'brawler': {'id': 16000019, 'name': 'PENNY', 'power': 11, 'trophies': 16}}, {'tag': '#92CRYR8JR', 'name': 'Adͥℝiͣaͫn🌎⚡', 'brawler': {'id': 16000054, 'name': 'FANG', 'power': 11, 'trophies': 16}}], [{'tag': '#PJJJR9JQ2', 'name': 'schwule gans', 'brawler': {'id': 16000015, 'name': 'PIPER', 'power': 11, 'trophies': 12}}, {'tag': '#YVQQJY0', 'name': 'мєℓιн™', 'brawler': {'id': 16000025, 'name': 'CARL', 'power': 11, 'trophies': 15}}, {'tag': '#9GYCJYGP', 'name': 'Armo', 'brawler': {'id': 16000064, 'name': 'GRAY', 'power': 11, 'trophies': 15}}]]"

# Regular expression to match the desired substrings
pattern = r"'brawler': \{[^}]+\}"

# Find all matches
matches = re.findall(pattern, input_string)

# Store matches in a list
brawler_list = matches

# Print the list
for brawler in brawler_list:
    print(brawler)

'brawler': {'id': 16000029, 'name': 'BEA', 'power': 11, 'trophies': 14}
'brawler': {'id': 16000019, 'name': 'PENNY', 'power': 11, 'trophies': 16}
'brawler': {'id': 16000054, 'name': 'FANG', 'power': 11, 'trophies': 16}
'brawler': {'id': 16000015, 'name': 'PIPER', 'power': 11, 'trophies': 12}
'brawler': {'id': 16000025, 'name': 'CARL', 'power': 11, 'trophies': 15}
'brawler': {'id': 16000064, 'name': 'GRAY', 'power': 11, 'trophies': 15}
